# 50 Plan 7 models (LOTO6)
N1..N6＋ALLをseries selectorとして作り、モデル台帳＆run_plan雛形を作る。

In [1]:
from fcast.meta.db import init_db, session_scope
from fcast.meta.orm import (RegistryLibrary, RegistryRelease, RegistrySymbol,
                            DatasetsDataset, DatasetsVersion, DatasetsGame,
                            FeaturesSet, FeaturesMaterialization, ModelsRegistry, ModelsExogSpec,
                            RunsDatasetBinding, RunsPlan, RunsStep, RunsParam)
from fcast.pipeline.planner import ensure_7_selectors_loto6
from fcast.lake.paths import table_path
init_db()


In [2]:
# Ensure game + dataset + version exist
with session_scope() as s:
    g = s.query(DatasetsGame).filter_by(game_code="loto6").one_or_none()
    if g is None:
        g = DatasetsGame(game_code="loto6", rules_json={"k":6,"maxv":43})
        s.add(g); s.flush()
    ds = s.query(DatasetsDataset).filter_by(name="loto6_demo").one_or_none()
    if ds is None:
        ds = DatasetsDataset(name="loto6_demo", game_id=g.game_id, domain="lottery")
        s.add(ds); s.flush()
    dv = s.query(DatasetsVersion).filter_by(dataset_id=ds.dataset_id).one_or_none()
    if dv is None:
        dv = DatasetsVersion(dataset_id=ds.dataset_id, source_snapshot={"type":"synthetic"}, lake_uri=str(table_path("gold","gold_loto6_features")))
        s.add(dv); s.flush()

    fs = s.query(FeaturesSet).filter_by(name="feat_basic").one_or_none()
    if fs is None:
        fs = FeaturesSet(name="feat_basic", purpose="train")
        s.add(fs); s.flush()

    # minimal registry entries for a mock model
    lib = s.query(RegistryLibrary).filter_by(name="mocklib").one_or_none()
    if lib is None:
        lib = RegistryLibrary(name="mocklib", vendor="internal")
        s.add(lib); s.flush()
    rel = s.query(RegistryRelease).filter_by(library_id=lib.library_id, version="0").one_or_none()
    if rel is None:
        rel = RegistryRelease(library_id=lib.library_id, version="0")
        s.add(rel); s.flush()
    sym = s.query(RegistrySymbol).filter_by(release_id=rel.release_id, qualname="mock.MockModel").one_or_none()
    if sym is None:
        sym = RegistrySymbol(release_id=rel.release_id, kind="class", qualname="mock.MockModel")
        s.add(sym); s.flush()

print("seed ok")


seed ok


In [3]:
selectors = ensure_7_selectors_loto6()
[(s.name, s.unique_id_list) for s in selectors]


DetachedInstanceError: Instance <ModelsSeriesSelector at 0x2e5fdc5e900> is not bound to a Session; attribute refresh operation cannot proceed (Background on this error at: https://sqlalche.me/e/20/bhk3)

In [ ]:
# Create 7 models + run plan (fit->predict->evaluate)
from datetime import datetime
with session_scope() as s:
    ds = s.query(DatasetsDataset).filter_by(name="loto6_demo").one()
    dv = s.query(DatasetsVersion).filter_by(dataset_id=ds.dataset_id).one()
    fs = s.query(FeaturesSet).filter_by(name="feat_basic").one()
    rel = s.query(RegistryRelease).join(RegistryLibrary).filter(RegistryLibrary.name=="mocklib").one()
    sym = s.query(RegistrySymbol).filter_by(release_id=rel.release_id, qualname="mock.MockModel").one()

    models = []
    for sel in selectors:
        m = ModelsRegistry(
            release_id=rel.release_id,
            model_symbol_id=sym.symbol_id,
            train_dataset_version_id=dv.dataset_version_id,
            feature_set_id=fs.feature_set_id,
            selector_id=sel.selector_id,
            series_mode="multivariate_joint" if sel.name.endswith("ALL") else "global_univariate",
            n_series=6 if sel.name.endswith("ALL") else 1,
            artifact_uri=f"artifacts/{sel.name}.pkl",
        )
        s.add(m); s.flush()
        s.add(ModelsExogSpec(model_id=m.model_id, hist_cols=["hist_y_diff1"], stat_cols=[], fut_cols=[]))
        models.append(m)

        bind = RunsDatasetBinding(dataset_version_id=dv.dataset_version_id, df_engine="pandas",
                                  feature_table_uri=dv.lake_uri)
        s.add(bind); s.flush()

        plan = RunsPlan(model_id=m.model_id, binding_id=bind.binding_id,
                        plan_name=f"demo_plan_{sel.name}",
                        pipeline_json={"steps":["fit","predict","evaluate"]})
        s.add(plan); s.flush()

        # steps
        fit = RunsStep(run_id=plan.run_id, step_name="fit", ord=0)
        pred = RunsStep(run_id=plan.run_id, step_name="predict", ord=1, depends_on_step_id=fit.run_step_id)
        eva = RunsStep(run_id=plan.run_id, step_name="evaluate", ord=2, depends_on_step_id=pred.run_step_id)
        s.add_all([fit,pred,eva]); s.flush()

        s.add_all([
            RunsParam(run_step_id=fit.run_step_id, params={"epochs":1}),
            RunsParam(run_step_id=pred.run_step_id, params={"h":5}),
            RunsParam(run_step_id=eva.run_step_id, params={"metric":"y_var"}),
        ])

print("created 7 models & plans")


In [ ]:
import pandas as pd
from fcast.meta.db import session_scope
from fcast.meta.orm import ModelsRegistry, RunsPlan
with session_scope() as s:
    ms = s.query(ModelsRegistry).all()
    rs = s.query(RunsPlan).all()
pd.DataFrame([{"model_id":m.model_id,"selector":m.selector_id,"mode":m.series_mode,"n_series":m.n_series} for m in ms]).head(), len(rs)
